In [1]:
from src.SB.soi_grad import SOI
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from pytorch_lightning.loggers import TensorBoardLogger
import numpy as np
from src.benchmark.toy_dataset import Task_redundant ,Task_synergy,Task_combination
from pytorch_lightning.trainer import seed_everything
seed_everything(33)


Seed set to 33


33

In [2]:
nb_mod = 10
dim=5
r = {} 

def get_samples(test_loader,mod_list,device):
        data ={ mod :torch.Tensor().to(device) for mod in mod_list} 
        for batch in test_loader:
            for mod in mod_list:
                data[mod]  = torch.cat([data[mod],batch[mod].to(device) ] )
        return data
    

def test_sgima():
    task = Task_combination(tasks= [Task_redundant(nb_var=3, rho=0.7 ,dim = dim),
                                    Task_synergy(nb_var=3, rho=0.6,dim = dim),
                                       Task_redundant(nb_var=3, rho=0.8,dim = dim),
                               ]  ,dim=dim)
    N =  1000 *100
    nb_mod = task.nb_var
    d_train, d_test = task.get_torch_dataset(N,10*1000,dim=dim,rescale=False)

    train_loader = DataLoader(d_train, batch_size=256,shuffle=True,
                                num_workers=8, drop_last=True)

    test_loader = DataLoader(d_test, batch_size=1000,
                                shuffle= False,
                                num_workers=8, drop_last=False)

    mod_list={ "x"+ str(i) : dim for i in range(nb_mod) }

    test_samples = get_samples(test_loader, mod_list=mod_list,device = "cuda")

    model = SOI(
                    test_samples= test_samples,
                    gt = task.get_summary(),
                    lr=1e-3,
                    mod_list =mod_list ,
                    use_ema= True, 
                    debias=True,
                    test_epoch= 10,
                    scores_order= 2,
                    debug= True,
                    margin_time=1,
                    tx=False,
                    fill_zeros= True,
                    weight_subsets=True )

    CHECKPOINT_DIR = "trained_models/"
    tb_logger =  TensorBoardLogger(save_dir = CHECKPOINT_DIR,name="test"+str(dim))
    trainer = pl.Trainer( logger= tb_logger,
                        accelerator='gpu', devices= 1,
                            max_epochs= 50, check_val_every_n_epoch=50,
                            default_root_dir = CHECKPOINT_DIR,
                        )

    trainer.fit(model=model, train_dataloaders=train_loader,val_dataloaders=test_loader  )
    return {"gt": task.get_summary(), "soi": model.compute_o_inf(test_samples, debias=True,nb_iter= 10)}
    


In [3]:
out = test_sgima()

starting to sample data
after cov
after reshape


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: trained_models/test5
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name      | Type | Params
-----------------------------------
0 | score     | DiT  | 1.5 M 
1 | model_ema | EMA  | 1.5 M 
-----------------------------------
3.0 M     Trainable params
0         Non-trainable params
3.0 M     Total params
11.972    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

O_inf -- GT: 2.6032749683598837   , e : -30.4765202999115
Gradient O_inf -- GT: [1.2188919712334183, 1.2188919712334183, 1.2188919712334183, -0.6195904097613933, -0.6195904097613294, -0.6195904097613365, 2.0039734068880293, 2.0039734068880293, 2.0039734068880293]   , e : {'x0': {'simple': 0.6893796920776367}, 'x1': {'simple': 0.34291791915893555}, 'x2': {'simple': 0.2134089469909668}, 'x3': {'simple': 0.543980598449707}, 'x4': {'simple': 0.3030414581298828}, 'x5': {'simple': 0.3804740905761719}, 'x6': {'simple': 0.2804443836212158}, 'x7': {'simple': 0.4767894744873047}, 'x8': {'simple': 15.3138427734375}}
O_inf -- GT: 2.6032749683598837   , e : -38.61995244026184
Gradient O_inf -- GT: [1.2188919712334183, 1.2188919712334183, 1.2188919712334183, -0.6195904097613933, -0.6195904097613294, -0.6195904097613365, 2.0039734068880293, 2.0039734068880293, 2.0039734068880293]   , e : {'x0': {'simple': 0.9642848968505859}, 'x1': {'simple': 0.8795380592346191}, 'x2': {'simple': 0.6002116203308105},